In [ ]:
import os

from astropy.io import fits
import astropy.table as at
import astropy.coordinates as coord
import astropy.units as u

import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm.notebook import tqdm
import thejoker as tj
import lightkurve as lk

import pymc3 as pm
import theano.tensor as tt
import exoplanet as xo

from hq.data import get_rvdata
from hq.samples_analysis import extract_MAP_sample

In [ ]:
kepid = 'KIC 11359305'
apid = '2M19435622+4907488'

In [ ]:
allvisit = fits.getdata('/mnt/home/apricewhelan/data/APOGEE_DR16/allVisit-r12-l33.fits')
binaries = at.QTable(at.Table.read('../catalogs/lnK0.0_logL4.6_metadata_gaia_starhorse.fits').filled())
row = binaries[binaries['APOGEE_ID'] == apid]

In [ ]:
visits = allvisit[allvisit['APOGEE_ID'] == apid]
rv_data = get_rvdata(visits)

In [ ]:
sample = extract_MAP_sample(row)
_ = tj.plot_phase_fold(data=rv_data, sample=sample)

In [ ]:
lcfs = lk.search_lightcurvefile(kepid, mission='Kepler').download_all()
stitched_lc = lcfs.PDCSAP_FLUX.stitch()

In [ ]:
# From DFM!
with lcfs[0].hdu as hdu:
    tpf_hdr = hdu[1].header
texp = tpf_hdr["FRAMETIM"] * tpf_hdr["NUM_FRM"]
texp /= 60.0 * 60.0 * 24.0  # days

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.plot(stitched_lc.astropy_time.tcb.jd,
        stitched_lc.flux,
        marker='o', ls='none', mew=0, ms=1.5, alpha=0.5)

MAD = np.nanmedian(np.abs(stitched_lc.flux - np.nanmedian(stitched_lc.flux)))
std = 1.5 * MAD
mask = (stitched_lc.flux - 1) < 4*std
ax.plot(stitched_lc.astropy_time.tcb.jd[~mask],
        stitched_lc.flux[~mask],
        marker='x', mew=2, ls='none', ms=8, color='r', zorder=10)

lc = stitched_lc[mask]

In [ ]:
# Convert to parts per thousand
x = lc.astropy_time.tcb.jd
y = lc.flux
mu = np.median(y)
y = (y / mu - 1) * 1e3
yerr = lc.flux_err * 1e3

x_ref = np.min(x)
x = x - x_ref

In [ ]:
len(x)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.plot(x, y,
        marker='o', ls='none', mew=0, ms=1.5, alpha=0.5)
ax.set_xlim(0, 500)

Radial velocity data, relative to the same reference time:

In [ ]:
x_rv = rv_data.t.tcb.jd - x_ref
y_rv = (rv_data.rv - row['MAP_v0'][0]).to_value(u.m/u.s)
yerr_rv = rv_data.rv_err.to_value(u.m/u.s)
mean_rv = np.mean(y_rv)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
phase = (lc.time / row['MAP_P'].value * 1.) % 1.
cc = ax.scatter(phase, lc.flux, c=lc.time,
                s=1, alpha=0.4, lw=1)
ax.set_xlabel('Phase')
ax.set_ylabel('Normalized Flux')
cb = fig.colorbar(cc)
cb.set_label('Kepler time', fontsize=14)
fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
from astropy.timeseries import BoxLeastSquares

m = np.zeros(len(x), dtype=bool)
period_grid = np.exp(np.linspace(np.log(sample['P'][0].value)-0.5, 
                                 np.log(sample['P'][0].value)+0.5, 
                                 10000))

bls = BoxLeastSquares(x[~m], y[~m])
bls_power = bls.power(period_grid, 0.1, oversample=20)

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
bls_depth = bls_power.depth[index]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.plot(x, y,
        marker='o', ls='none', mew=0, ms=1.5, alpha=0.5)
ax.set_xlim(0, 200)
for i in range(10):
    ax.axvline(bls_t0 + i*bls_period, color='tab:red', alpha=0.25)

In [ ]:
results = xo.estimators.lomb_scargle_estimator(
    x, y, max_peaks=1, min_period=10.0, max_period=100.0, samples_per_peak=50
)

peak = results["peaks"][0]
freq, power = results["periodogram"]
plt.plot(1 / freq, power, color="k", marker='')
plt.axvline(peak["period"], color="k", lw=4, alpha=0.3)
plt.xlim(1, 100)
plt.xlabel("period [days]")
plt.ylabel("power");

est_rot_period = peak["period"]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.plot(x, y,
        marker='o', ls='none', mew=0, ms=1.5, alpha=0.5)
ax.set_xlim(0, 200)
for xxx in np.arange(0, 10) * est_rot_period:
    ax.axvline(xxx, color='tab:red', alpha=0.25)

Load primary stellar radius and mass from STARHORSE and Gaia:

In [ ]:
M_star = row['mass'][0], row['mass_err'][0]
R_star = row['radius_val'][0], row['radius_val'][0] - row['radius_percentile_lower'][0]
M_star, R_star

In [ ]:
df = 1 - 40/1e3
R2 = R_star[0] * np.sqrt(df)
R2

In [ ]:
msini = row['m2_min_50'][0].to(u.M_sun)
msini, msini.to(u.Mjup)

In [ ]:
rv_span = np.max(x_rv) - np.min(x_rv)
t_rv = np.arange(x_rv.min() - rv_span/10, 
                 x_rv.max() + rv_span/10, 
                 row['MAP_P'][0].to_value(u.day) / 128)


def build_model(phot_mask=None, start=None):
    if phot_mask is None:
        phot_mask = np.ones(len(x), dtype=bool)
        
    with pm.Model() as model:

        # Parameters for the stellar properties of the primary
        u_star = xo.distributions.QuadLimbDark("u_star")
        BoundedNormal = pm.Bound(pm.Normal, lower=0.5, upper=3)
        m_star = BoundedNormal("m_star", mu=M_star[0], sd=M_star[1])
        r_star = BoundedNormal("r_star", mu=R_star[0], sd=R_star[1])

        # Parameters of the companion
        logm = pm.Normal("logm", mu=np.log(msini.value), sd=1)  # companion mass
        logr = pm.Normal(
            "logr",
            mu=0.5 * np.log(1e-3 * bls_depth) + np.log(R_star[0]),
            sd=1,
        )  # companion radius
        r_pl = pm.Deterministic("r_pl", tt.exp(logr))
        rpl_over_rstar = pm.Deterministic("ror", r_pl / r_star)
        b = xo.distributions.ImpactParameter("b", ror=rpl_over_rstar)
        m_pl = pm.Deterministic("m_pl", tt.exp(logm))  # for tracking
        
        # Parameters of the orbit
        logP = pm.Normal("logP", mu=np.log(bls_period), sd=1)
        t0 = pm.Normal("t0", mu=bls_t0, sd=1)
        ecc = xo.distributions.eccentricity.kipping13(
            "ecc", long=False, testval=0.043
        )
        omega = xo.distributions.Angle("omega")
        period = pm.Deterministic("period", tt.exp(logP))  # for tracking

        # RV jitter
        logs_rv = pm.Normal("logs_rv", mu=np.log(500.), sd=0.5)  # MAP_s~350 m/s, but prob. bigger
        rv0 = pm.Normal("rv0", mu=0, sd=500.)  # m/s

        # Transit jitter & GP parameters
        logs2 = pm.Normal("logs2", mu=np.log(np.var(y[phot_mask])), sd=2)
        logw0 = pm.Normal("logw0", mu=0.0, sd=3)
        logSw4 = pm.Normal("logSw4", mu=np.log(np.var(y[phot_mask])), sd=4)
        
        # The parameters of the RotationTerm kernel
        logamp = pm.Normal("logamp", mu=np.log(np.var(y[phot_mask])), sd=5.0)
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0, upper=np.log(50))
        logrotperiod = BoundedNormal("logrotperiod", mu=np.log(est_rot_period), sd=5.0)
        logQ0 = pm.Normal("logQ0", mu=1.0, sd=10.0)
        logdeltaQ = pm.Normal("logdeltaQ", mu=2.0, sd=10.0)
        mix = xo.distributions.UnitUniform("mix")
        rotperiod = pm.Deterministic("rotperiod", tt.exp(logrotperiod))

        # Orbit model
        orbit = xo.orbits.KeplerianOrbit(
            r_star=r_star,
            m_star=m_star,
            period=period,
            t0=t0,
            b=b,
            m_planet=xo.units.with_unit(m_pl, msini.unit),
            ecc=ecc,
            omega=omega,
        )

        # Compute the transit light curve using starry
        mean_flux = pm.Normal("mean_flux", mu=0.0, sd=10.0)
        transit_light_curves = xo.LimbDarkLightCurve(u_star).get_light_curve(
            orbit=orbit, r=r_pl, t=x[phot_mask], texp=texp) * 1e3
        
        transit_light_curve = pm.math.sum(transit_light_curves, axis=-1) + mean_flux
        # transit_light_curve = transit_light_curves + mean_flux
        pm.Deterministic("transit_light_curves", transit_light_curves)

        # GP model for the light curve
        # Set up the Gaussian Process model
        kernel = xo.gp.terms.SHOTerm(log_Sw4=logSw4, log_w0=logw0, Q=1 / np.sqrt(2))
        kernel += xo.gp.terms.RotationTerm(
            log_amp=logamp, period=rotperiod, 
            log_Q0=logQ0, log_deltaQ=logdeltaQ, mix=mix
        )
        gp = xo.gp.GP(kernel, x[phot_mask], 
                      yerr[phot_mask]**2 + tt.exp(logs2))
        pm.Potential("transit_obs", gp.log_likelihood(y[phot_mask] - transit_light_curve))
        pm.Deterministic("gp_pred", gp.predict())

        # Set up the RV model and save it as a deterministic
        # for plotting purposes later
        vrad = orbit.get_radial_velocity(x_rv)
        pm.Deterministic("vrad", vrad)

        # The likelihood for the RVs
        # rv_model = pm.Deterministic("rv_model", tt.sum(vrad, axis=-1) + rv0)
        rv_model = pm.Deterministic("rv_model", vrad + rv0)
        err = tt.sqrt(yerr_rv ** 2 + tt.exp(2 * logs_rv))
        pm.Normal("obs", mu=rv_model, sd=err, observed=y_rv)

        # vrad_pred = orbit.get_radial_velocity(t_rv)
        vrad_pred = orbit.get_radial_velocity(t_rv)
        pm.Deterministic("vrad_pred", vrad_pred)
        bkg_pred = pm.Deterministic("bkg_pred", rv0)
        pm.Deterministic("rv_model_pred", vrad_pred + bkg_pred)

        # Fit for the maximum a posteriori parameters, I've found that I can get
        # a better solution by trying different combinations of parameters in turn
        if start is None:
            start = model.test_point
        map_soln = xo.optimize(start=start, vars=[logP, t0])
        map_soln = xo.optimize(start=map_soln, vars=[logs2, logs_rv])
        map_soln = xo.optimize(start=map_soln, vars=[logs2, logSw4])
        map_soln = xo.optimize(start=map_soln, vars=[logw0])
        # map_soln = xo.optimize(start=map_soln, vars=[logr, b])
        map_soln = xo.optimize(start=map_soln, vars=[logP, t0, omega, ecc, rv0, logs_rv])
        map_soln = xo.optimize(start=map_soln)

    return model, map_soln

In [ ]:
sub_mask = np.zeros(len(x), dtype=bool)
sub_mask[:15000] = True
sub_mask &= np.isfinite(y)

In [ ]:
model0, map_soln0 = build_model(phot_mask=sub_mask)

In [ ]:
y_fontsize = 12

def plot_light_curve(soln, mask=None):
    if mask is None:
        mask = np.ones(len(x), dtype=bool)

    fig, axes = plt.subplots(4, 1, figsize=(12, 12), sharex=True, sharey=True)

    ax = axes[0]
    ax.plot(x[mask], y[mask], "k", label="data")
    gp_mod = soln["gp_pred"] + soln["mean_flux"]
    ax.plot(x[mask], gp_mod, color="C2", label="gp model")
    ax.legend(fontsize=10)
    ax.set_ylabel("relative flux [ppt]", fontsize=y_fontsize)

    ax = axes[1]
    ax.plot(x[mask], y[mask] - gp_mod, "k")
    ax.set_ylabel("de-trended flux [ppt]", fontsize=y_fontsize)
    
    ax = axes[2]
    ax.plot(x[mask], soln["transit_light_curves"][:, 0], "k")
    ax.set_ylabel("transit [ppt]", fontsize=y_fontsize)

    ax = axes[3]
    mod = gp_mod + soln["transit_light_curves"][:, 0]
    ax.plot(x[mask], y[mask] - mod, "k")
    ax.axhline(0, color="#aaaaaa", lw=1)
    ax.set_ylabel("residuals [ppt]", fontsize=y_fontsize)
    ax.set_xlim(x[mask].min(), x[mask].max())
    ax.set_xlabel("time [days]")
    
    fig.tight_layout()

    return fig

def plot_rv_curve(soln):
    err = np.sqrt(yerr_rv ** 2 + np.exp(2 * soln["logs_rv"]))
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 7))

    ax = axes[0]
    ax.errorbar((x_rv/soln['period']) % 1, y_rv, yerr=yerr_rv, 
                marker='o', ls='none', color='k')
    ax.errorbar((x_rv/soln['period']) % 1, y_rv, yerr=err, 
                marker='', ls='none', color='tab:red', zorder=-100)
    ax.plot((t_rv/soln['period']) % 1, soln["vrad_pred"], "--k", alpha=0.5, color='tab:blue')
    ax.plot((t_rv/soln['period']) % 1, soln["rv_model_pred"], label="model")
    ax.legend(fontsize=10)
    ax.set_ylabel("radial velocity [m/s]", fontsize=y_fontsize)
    ax.set_xlim(0, 1)

    ax = axes[1]
    ax.errorbar(x_rv, y_rv - soln["rv_model"], yerr=yerr_rv, 
                marker='o', ls='none', color='k')
    ax.errorbar(x_rv, y_rv - soln["rv_model"], yerr=err, 
                marker='', ls='none', color='tab:red', zorder=-100)
    ax.axhline(0, color="k", lw=1)
    ax.set_ylabel("residuals [m/s]", fontsize=y_fontsize)
    ax.set_xlim(t_rv.min(), t_rv.max())
    ax.set_xlabel("time [days]")
    fig.tight_layout()

In [ ]:
fig = plot_light_curve(map_soln0, mask=sub_mask)
fig.axes[0].set_xlim(0, 300)
fig.axes[0].set_ylim(-10, 5)
fig.set_facecolor('w')

In [ ]:
plot_rv_curve(map_soln0)

Sigma clip the outliers:

In [ ]:
_mask = sub_mask

mod = (
    map_soln0["gp_pred"]
    + map_soln0["mean_flux"]
    + np.sum(map_soln0["transit_light_curves"], axis=-1)
)
resid = y[_mask] - mod
rms = np.sqrt(np.median(resid ** 2))
mask_iter1 = np.abs(resid) < 10 * rms

plt.plot(x[_mask], resid, "k", label="data")
plt.plot(x[_mask][~mask_iter1], resid[~mask_iter1], "xr", mew=1, label="outliers")
plt.axhline(0, color="#aaaaaa", lw=1)
plt.ylabel("residuals [ppt]")
plt.xlabel("time [days]")
plt.legend(fontsize=12, loc=4)
plt.xlim(x[_mask].min(), x[_mask].max());
plt.axhline(7*rms)
plt.axhline(-7*rms)

In [ ]:
tmp_mask = sub_mask.copy()
tmp_mask[:len(mask_iter1)] = mask_iter1
model, map_soln1 = build_model(start=map_soln0, phot_mask=tmp_mask)

In [ ]:
fig = plot_light_curve(map_soln1, tmp_mask);
fig.axes[0].set_xlim(0, 300)
fig.axes[0].set_ylim(-10, 5)
fig.set_facecolor('w')

In [ ]:
plot_rv_curve(map_soln1)

In [ ]:
np.exp(map_soln1['logP'])

In [ ]:
map_soln1['ecc']

In [ ]:
map_soln1['r_pl']

In [ ]:
mass = map_soln1['m_pl'] * u.Msun
mass.to(u.Mjup)

In [ ]:
map_soln1['rotperiod']

---

In [ ]:
np.random.seed(123)
with model:
    trace = pm.sample(
        tune=1000,
        draws=1000,
        start=map_soln1,
        chains=4,
        step=xo.get_dense_nuts_step(target_accept=0.9),
    )

In [ ]:
pm.summary(trace, var_names=["period", "r_pl", "m_pl", "ecc", "omega", "b", "rotperiod"])

In [ ]:
print(trace.varnames)

In [ ]:
np.mean((trace['r_pl'] * u.Rsun).to(u.Rjup))

In [ ]:
(np.mean(trace['m_pl'])*u.Msun).to(u.Mjup), (np.std(trace['m_pl'])*u.Msun).to(u.Mjup)

In [ ]:
np.mean((trace['rotperiod'] * u.day))

In [ ]:
np.mean(trace['ecc'])

In [ ]:
np.mean(trace['m_star'])

In [ ]:
plt.figure(figsize=(8, 5))

# Get the posterior median orbital parameters
p = np.median(trace["period"])
t0 = np.median(trace["t0"])

# Plot the folded data
x_fold = (x_rv - t0 + 0.5 * p) % p - 0.5 * p
plt.errorbar(x_fold, y_rv, yerr=yerr_rv, fmt=".k", label="data")

# Compute the posterior prediction for the folded RV model for this
# planet
t_fold = (t_rv - t0 + 0.5 * p) % p - 0.5 * p
inds = np.argsort(t_fold)
pred = np.percentile(trace["vrad_pred"][:, inds], [16, 50, 84], axis=0)
plt.plot(t_fold[inds], pred[1], color="C1", label="model", marker='')
art = plt.fill_between(t_fold[inds], pred[0], pred[2], color="C1", alpha=0.3)
art.set_edgecolor("none")

plt.legend(fontsize=10)
plt.xlim(-0.5 * p, 0.5 * p)
plt.xlabel("phase [days]")
plt.ylabel("radial velocity [m/s]")
# plt.title("K2-24{0}".format(letter));

In [ ]:
plt.figure(figsize=(8, 5))

# Compute the GP prediction
gp_mod = np.median(trace["gp_pred"] + trace["mean_flux"][:, None], axis=0)

# Get the posterior median orbital parameters
p = np.median(trace["period"])
t0 = np.median(trace["t0"])

# Plot the folded data
x_fold = (x[tmp_mask] - t0 + 0.5 * p) % p - 0.5 * p
plt.plot(x_fold, y[tmp_mask] - gp_mod, ".k", label="data", zorder=-1000)

# Plot the folded model
inds = np.argsort(x_fold)
inds = inds[np.abs(x_fold)[inds] < 0.3]
pred = trace["transit_light_curves"][:, inds, 0]
pred = np.percentile(pred, [16, 50, 84], axis=0)
plt.plot(x_fold[inds], pred[1], color="C1", label="model", marker='')
art = plt.fill_between(
    x_fold[inds], pred[0], pred[2], color="C1", alpha=0.5, zorder=1000
)
art.set_edgecolor("none")

# Annotate the plot with the planet's period
txt = "period = {0:.4f} +/- {1:.4f} d".format(
    np.mean(trace["period"]), np.std(trace["period"])
)

plt.xlim(-0.5 * p, 0.5 * p)
plt.xlabel("time since transit [days]")
plt.ylabel("de-trended flux")
# plt.title("K2-24{0}".format(letter))
plt.xlim(-0.3, 0.3)